In [1]:
import pandas as pd
f = open("c:/data/text/news1.txt", 'r', encoding='utf-8')
lines = f.readlines()
f.close()


In [2]:
import re
from konlpy.tag import Hannanum
han = Hannanum()
word_list = []
for i in range(len(lines)):
    sentence =re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', lines[i])
    #명사만 추출
    a=sentence.strip()
    n=han.nouns(a)
    n2=[x for x in n if len(x)>1]
    if len(n2)>0:
        word_list.append(n2)

In [3]:
# tf-idf 행렬을 만들기 위해 다시 역토큰화
detokenized_doc = []
for i in range(len(word_list)):
    t = ' '.join(word_list[i])
    detokenized_doc.append(t)
detokenized_doc

['서부컨퍼런스 챔피언 골든스테이트 워리어스 파이널',
 '샴스 카라니아 기자 골든스테이트 스테픈 커리 가드 이하 한국시간 열리 보스턴 셀틱스와 파이널 차전 출장 긍정적',
 '커리 파이널 차전 부상 경기 종료 시점 커리 선수 소유권 과정 이후 시즌 막판 부상 자리 이번 부상 차전 출장 의문 거론',
 '심각 부상 확인 정밀검사 필요치 상황 차전 차전 사이 휴식 하루 불과 부분 골든스테이트 입장 차전 시리즈 주도권 가운데 커리 잔부상 완연 상태 예상 이대 분위기',
 '이날 점슛 포함 이날 리바운드 어시스트 스틸 골든스테이트 에이스 맹활약 이번 부상 시리즈 컨디션 유지 차질 하루 경기력 골든스테이트 입장 중요',
 '골든스테이트 이날 커리 득점 가운데 클레이 탐슨 점슛 리바운드 어시스트 앤드 위긴스 리바운드 어시스트 스틸 블록 기록 선수층 골든스테이트 벤치 득점 지원 부족',
 '결정적 드레이먼드 그린 파울아웃 그린 골든스테이트 골밑 인사이드 로테이션 보스턴 상대적 여의치 골든스테이트 그린 반칙 누적 부담 수비 전열 구축 공격 경기 운영 악영향 야기',
 '커리 이번 시리즈 경기 평균 리바운드 어시스트 스틸 기록 효율 자랑 골든스테이트 공격 주도하 시리즈 평균 점슛 육박 성공률 골든스테이트 맹활약 보스턴 상황',
 '기사제공 바스켓코리',
 '이재승']

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(detokenized_doc)
X.shape # TF-IDF 행렬의 크기 확인


(10, 105)

In [5]:
from sklearn.decomposition import TruncatedSVD
# n_components 토픽수
svd_model = TruncatedSVD(n_components=17)
svd_model.fit(X)
len(svd_model.components_)

10

In [6]:
import numpy as np
# x 토픽수 단어수
np.shape(svd_model.components_)

(10, 105)

In [7]:
svd_model.components_

array([[ 0.05496305,  0.10130015,  0.05210376, ...,  0.05291059,
         0.08155699,  0.05291059],
       [ 0.1186611 , -0.00815411,  0.10020421, ...,  0.06878696,
        -0.05738174,  0.06878696],
       [ 0.07022573, -0.06890474, -0.04431517, ..., -0.0721508 ,
         0.01040508, -0.0721508 ],
       ...,
       [-0.05444806, -0.13306965,  0.17652446, ..., -0.15455255,
        -0.0078846 , -0.15455255],
       [-0.01198677,  0.16839326,  0.02195441, ...,  0.02719119,
        -0.1821368 ,  0.02719119],
       [ 0.07450533, -0.08535022, -0.04792423, ..., -0.01441896,
        -0.12739421, -0.01441896]])

In [8]:
# 단어 집합
terms = vectorizer.get_feature_names()
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i
in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)
#각 토픽의 핵심 키워드 추출

Topic 1: [('골든스테이트', 0.33846), ('차전', 0.23643), ('시리즈', 0.22639), ('어시스트', 0.22493), ('리바운드', 0.22493)]
Topic 2: [('차전', 0.39078), ('부상', 0.22509), ('파이널', 0.22076), ('출장', 0.18606), ('커리', 0.1497)]
Topic 3: [('그린', 0.33388), ('워리어스', 0.29811), ('챔피언', 0.29811), ('서부컨퍼런스', 0.29811), ('파이널', 0.24098)]
Topic 4: [('이재승', 0.91154), ('바스켓코리', 0.29078), ('기사제공', 0.29078), ('평균', 0.0), ('어시스트', 0.0)]
Topic 5: [('기사제공', 0.64455), ('바스켓코리', 0.64455), ('득점', 0.0), ('가운데', 0.0), ('벤치', 0.0)]
Topic 6: [('그린', 0.41254), ('공격', 0.1376), ('결정적', 0.13751), ('악영향', 0.13751), ('반칙', 0.13751)]
Topic 7: [('가드', 0.22426), ('열리', 0.22426), ('한국시간', 0.22426), ('스테픈', 0.22426), ('셀틱스와', 0.22426)]
Topic 8: [('부상', 0.3083), ('거론', 0.17652), ('과정', 0.17652), ('선수', 0.17652), ('막판', 0.17652)]
Topic 9: [('득점', 0.34179), ('앤드', 0.1709), ('선수층', 0.1709), ('블록', 0.1709), ('위긴스', 0.1709)]
Topic 10: [('이날', 0.31753), ('컨디션', 0.22975), ('에이스', 0.22975), ('포함', 0.22975), ('중요', 0.22975)]


c:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
